<a href="https://colab.research.google.com/github/fran53759/nba_games_prediction/blob/modelVersions/neumreProjekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NBA games prediction project**

In [1]:
# Imports
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import pandas as pd
import numpy as np

### 1) DATA COLLECTION

In [2]:
# Function for getting all regular season games from specified season
def download_season(season):
    print(f"Downloading season {season}...")
    games = leaguegamefinder.LeagueGameFinder(
            season_nullable=season,
            season_type_nullable='Regular Season'
        )
    df = games.league_game_finder_results.get_data_frame()
    return df

# Note: Train games must be games held before test games 
train_games_df = pd.concat([download_season('2022-23'),
                            download_season('2023-24')],
                            ignore_index=True)
test_games_df = download_season('2024-25')

print(f'Downloaded {len(train_games_df)} train games and {len(test_games_df)} test games!')

train_games_df.head()

Downloaded 4920 train games and 2460 test games!


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612739,CLE,Cleveland Cavaliers,0022201218,2023-04-09,CLE vs. CHA,L,241,95,...,0.938,11,33,44,27,9,5,16,24,-11.0
1,22022,1610612763,MEM,Memphis Grizzlies,0022201226,2023-04-09,MEM @ OKC,L,241,100,...,0.722,11,32,43,25,8,4,12,16,-15.0
2,22022,1610612743,DEN,Denver Nuggets,0022201227,2023-04-09,DEN vs. SAC,W,240,109,...,0.720,15,36,51,25,11,2,16,15,14.0
3,22022,1610612748,MIA,Miami Heat,0022201219,2023-04-09,MIA vs. ORL,W,241,123,...,0.750,7,37,44,30,10,3,18,20,13.0
4,22022,1610612752,NYK,New York Knicks,0022201220,2023-04-09,NYK vs. IND,L,241,136,...,0.773,19,34,53,29,8,8,15,24,-5.0


In [3]:
def adjust_game_df(games_df):
    # Adding binary "WIN" column
    games_df["WIN"] = games_df["WL"].apply(lambda x: 1 if x == 'W' else 0)

    # Adding "HGA" (Home game advantage) column
    games_df["HGA"] = games_df["MATCHUP"].apply(lambda x: 1 if 'vs' in x else 0)

    # Converting int stat columns to float type
    int_columns = ['MIN','PTS','FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','DREB','REB','AST','STL','BLK','TOV','PF','HGA']
    games_df[int_columns] = games_df[int_columns].astype(float)

    # Convert game date to pandas datetime
    games_df["GAME_DATE"] = pd.to_datetime(games_df["GAME_DATE"])

    # Sorting games by game_date
    games_df = games_df.sort_values(by='GAME_DATE').reset_index(drop=True)

    return games_df

train_games_df = adjust_game_df(train_games_df)
test_games_df = adjust_game_df(test_games_df)

train_games_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,WIN,HGA
0,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240.0,126.0,...,30.0,36.0,24.0,8.0,3.0,10.0,24.0,9.0,1,1.0
1,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239.0,117.0,...,27.0,31.0,16.0,8.0,3.0,14.0,25.0,-9.0,0,0.0
2,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241.0,109.0,...,39.0,48.0,23.0,12.0,4.0,21.0,18.0,-14.0,0,0.0
3,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241.0,123.0,...,37.0,48.0,31.0,11.0,4.0,18.0,23.0,14.0,1,1.0
4,22022,1610612758,SAC,Sacramento Kings,0022200014,2022-10-19,SAC vs. POR,L,241.0,108.0,...,37.0,41.0,27.0,8.0,5.0,15.0,25.0,-7.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,22023,1610612751,BKN,Brooklyn Nets,0022301192,2024-04-14,BKN @ PHI,L,240.0,86.0,...,38.0,42.0,19.0,8.0,6.0,12.0,14.0,-21.0,0,0.0
4916,22023,1610612759,SAS,San Antonio Spurs,0022301197,2024-04-14,SAS vs. DET,W,241.0,123.0,...,42.0,54.0,33.0,9.0,3.0,13.0,17.0,28.0,1,1.0
4917,22023,1610612752,NYK,New York Knicks,0022301190,2024-04-14,NYK vs. CHI,W,265.0,120.0,...,37.0,53.0,27.0,7.0,6.0,21.0,17.0,1.0,1,1.0
4918,22023,1610612745,HOU,Houston Rockets,0022301199,2024-04-14,HOU @ LAC,W,240.0,116.0,...,42.0,59.0,31.0,7.0,8.0,18.0,13.0,11.0,1,0.0


### 2) FEATURE ENGINEERING

*-> Functions used for extracting feautures*

In [29]:
# Calculates win rate difference 
def win_rate_diff(team_recent_games, opponent_recent_games, game):
    return team_recent_games["WIN"].mean() - opponent_recent_games["WIN"].mean()

# Returns 1 if team has home advantage
def home_advantage(team_recent_games, opponent_recent_games, game):
    return game["HGA"]

# Calculates difference in average points scored between teams
def avg_points_diff(team_recent_games, opp_recent_games, game):
    return team_recent_games["PTS"].mean() - opp_recent_games["PTS"].mean()

# Calculates difference in three-point shooting percentage
def fg3_pct_diff(team_recent_games, opp_recent_games, game):
    team_pct = team_recent_games["FG3M"].sum() / max(team_recent_games["FG3A"].sum(), 1)
    opp_pct = opp_recent_games["FG3M"].sum() / max(opp_recent_games["FG3A"].sum(), 1)
    return team_pct - opp_pct

# Calculates difference in overall field goal percentage (2PT + 3PT)
def fg_pct_diff(team_recent_games, opp_recent_games, game):
    return (
        team_recent_games["FGM"].sum() / team_recent_games["FGA"].sum()
        - opp_recent_games["FGM"].sum() / opp_recent_games["FGA"].sum()
    )

# Calculates difference in average blocks per game
def blocks_diff(team_recent_games, opp_recent_games, game):
    return team_recent_games["BLK"].mean() - opp_recent_games["BLK"].mean()

# Calculates difference in average rebounds per game
def rebounds_diff(team_recent_games, opp_recent_games, game):
    return team_recent_games["REB"].mean() - opp_recent_games["REB"].mean()

# Calculates difference in average assists per game
def assists_diff(team_recent, opp_recent, game):
    return team_recent["AST"].mean() - opp_recent["AST"].mean()

# Calculates difference in average turnovers per game
def turnovers_diff(team_recent, opp_recent, game):
    return opp_recent["TOV"].mean() - team_recent["TOV"].mean()

FEATURE_FUNCTIONS = {  "win_rate_diff": win_rate_diff,
    "home_advantage": home_advantage,
    "avg_points_diff": avg_points_diff,
    "fg3_pct_diff": fg3_pct_diff,
    "fg_pct_diff": fg_pct_diff,
    "blocks_diff": blocks_diff,
    "rebounds_diff": rebounds_diff,
    "assists_diff" : assists_diff,
    "turnovers_diff": turnovers_diff,
    }

In [32]:
# Function that returns label and list of calculated features for specific game
def extract_features(game, all_games, feature_list, n_matches = 10):

    team_id = game["TEAM_ID"]
    opponent_id = get_opponent_id(game)
    game_date = game["GAME_DATE"]

    # Find last n games played by both teams
    team_recent_games = all_games[(all_games['TEAM_ID'] == team_id) & (all_games['GAME_DATE'] < game_date)].tail(n_matches)
    opponent_recent_games = all_games[(all_games['TEAM_ID'] == opponent_id) & (all_games['GAME_DATE'] < game_date)].tail(n_matches)

    # If there isn't enough information skip
    if len(team_recent_games) < n_matches or len(opponent_recent_games) < n_matches:
        return None

    # Generate features by callig feature functions defined in FEATURE_FUNCTIONS
    features = [
        FEATURE_FUNCTIONS[name](
            team_recent_games,
            opponent_recent_games,
            game
        )
        for name in feature_list
    ]

    label = game["WIN"]

    return label, features

# Function that returns opponent team id
def get_opponent_id(game):
    matchup = game["MATCHUP"]
    if 'vs' in matchup:
        opponent_abbreviation = matchup.split(' vs. ')[-1]
    else:
        opponent_abbreviation = matchup.split(' @ ')[-1]

    opponent_team = teams.find_team_by_abbreviation(opponent_abbreviation)
    return opponent_team["id"]

# Function that creates and returns dataset X and labels y
def create_datasets(games_df: pd.DataFrame, feature_list, n_matches = 10, print_progress = False):

    X = []
    y = []

    skipped = 0
    num_of_games = len(games_df)

    for idx, game in games_df.iterrows():
        result = extract_features(game, games_df, feature_list, n_matches)

        if result is None:
            skipped += 1
            continue

        label, features = result

        X.append(features)
        y.append(label)

        if (idx + 1) % 500 == 0 and print_progress:
            print(f"Processed {idx + 1}/{num_of_games} games...")

    if print_progress:
        print(f"Created dataset with {num_of_games - skipped} games.")
        print(f"Skipped {skipped} games (games with less than {n_matches} previous matches).\n")

    return np.array(X), np.array(y)

default_feature_list = ['win_rate_diff', 'home_advantage', 'avg_points_diff', 'fg_pct_diff']

X_train, y_train = create_datasets(train_games_df, default_feature_list, print_progress=True)
X_test, y_test = create_datasets(test_games_df, default_feature_list, print_progress=True)

Processed 500/4920 games...
Processed 1000/4920 games...
Processed 1500/4920 games...
Processed 2000/4920 games...
Processed 2500/4920 games...
Processed 3000/4920 games...
Processed 3500/4920 games...
Processed 4000/4920 games...
Processed 4500/4920 games...
Created dataset with 4606 games.
Skipped 314 games (games with less than 10 previous matches).

Processed 500/2460 games...
Processed 1000/2460 games...
Processed 1500/2460 games...
Processed 2000/2460 games...
Created dataset with 2150 games.
Skipped 310 games (games with less than 10 previous matches).



### 3) BASELINE MODEL (Logistic Regression)

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

predictions_lr = log_reg.predict(X_test_scaled)
prediction_prob_lr = log_reg.predict_proba(X_test_scaled)[:, 1]

print(f"LR Train Accuracy = {accuracy_score(y_train, log_reg.predict(X_train_scaled))}")
print(f"LR Test Accuracy  = {accuracy_score(y_test, predictions_lr)}")

df_lr = pd.DataFrame({
    "y_test": y_test,
    "predicted": predictions_lr,
    "winning_probability": prediction_prob_lr
})
display(df_lr.head())



LR Train Accuracy = 0.6048632218844985
LR Test Accuracy  = 0.633953488372093


,y_test,predicted,winning_probability
0,0,0,0.393439
1,1,1,0.606561
2,0,1,0.545958
3,1,0,0.454042
4,0,0,0.357118


#### 3.1) Evaluation of different features and their predictive significance (with baseline model)

In [39]:
n_matches_list = [2, 5, 10]

FEATURE_GROUPS = {
    "1" : ["home_advantage", "win_rate_diff"],
    "2": ["home_advantage", "win_rate_diff", "fg_pct_diff", "avg_points_diff"],
    "3": ["win_rate_diff", "fg_pct_diff", "avg_points_diff"],
    "4": ["home_advantage", "win_rate_diff", "fg3_pct_diff", "rebounds_diff", "turnovers_diff"],
    "5": ["home_advantage", "win_rate_diff", "fg3_pct_diff", "rebounds_diff", "turnovers_diff", "avg_points_diff"],
    "6": ["rebounds_diff", "fg_pct_diff", "avg_points_diff"]
}

results = []

for n_matches in n_matches_list:
    for name, feature_list in FEATURE_GROUPS.items():

        # Creating datasets
        X_train, y_train = create_datasets(train_games_df, feature_list, n_matches)
        X_test, y_test = create_datasets(test_games_df, feature_list, n_matches)

        # Scaling features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Logistic Regression
        log_reg = LogisticRegression(max_iter=1000)
        log_reg.fit(X_train_scaled, y_train)

        acc_train = accuracy_score(y_train, log_reg.predict(X_train_scaled))
        acc_test = accuracy_score(y_test, log_reg.predict(X_test_scaled))

        results.append({
            "n_matches": n_matches,
            "feature_list": feature_list,
            "train_accuracy": acc_train,
            "test_accuracy": acc_test
        })

pd.set_option("display.max_colwidth", None)

results_df = pd.DataFrame(results).reset_index(drop=True)
display(results_df.sort_values("test_accuracy", ascending=False).style.hide(axis="index"))

pd.reset_option("display.max_colwidth")

n_matches,feature_list,train_accuracy,test_accuracy
10,"['win_rate_diff', 'fg_pct_diff', 'avg_points_diff']",0.606166,0.648837
10,"['home_advantage', 'win_rate_diff', 'fg_pct_diff', 'avg_points_diff']",0.604863,0.633953
5,"['home_advantage', 'win_rate_diff', 'fg_pct_diff', 'avg_points_diff']",0.593868,0.629340
10,"['home_advantage', 'win_rate_diff', 'fg3_pct_diff', 'rebounds_diff', 'turnovers_diff']",0.606600,0.627442
10,"['home_advantage', 'win_rate_diff', 'fg3_pct_diff', 'rebounds_diff', 'turnovers_diff', 'avg_points_diff']",0.607903,0.627442
5,"['home_advantage', 'win_rate_diff', 'fg3_pct_diff', 'rebounds_diff', 'turnovers_diff']",0.587148,0.626736
5,"['home_advantage', 'win_rate_diff', 'fg3_pct_diff', 'rebounds_diff', 'turnovers_diff', 'avg_points_diff']",0.591348,0.625868
10,"['home_advantage', 'win_rate_diff']",0.612245,0.620930
5,"['home_advantage', 'win_rate_diff']",0.594708,0.619792
5,"['win_rate_diff', 'fg_pct_diff', 'avg_points_diff']",0.567409,0.614583


### 4) NEURAL NETWORK MODEL (MLP)

In [ ]:
# MLP
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, log_loss

def _get_activation(name: str) -> nn.Module:
    name = name.lower()
    if name == "relu":
        return nn.ReLU()
    if name == "leakyrelu":
        return nn.LeakyReLU(0.01)
    if name == "tanh":
        return nn.Tanh()
    if name == "sigmoid":
        return nn.Sigmoid()
    raise ValueError(f"Nepoznata aktivacija: {name} (probaj: relu, leakyrelu, tanh, sigmoid)")

def _get_optimizer(name: str, params, lr: float, weight_decay: float):
    name = name.lower()
    if name == "adam":
        return torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)
    if name == "sgd":
        return torch.optim.SGD(params, lr=lr, momentum=0.9, weight_decay=weight_decay)
    if name == "adamw":
        return torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)
    raise ValueError(f"Nepoznat optimizer: {name} (probaj: adam, sgd, adamw)")

class MLP(nn.Module):
    def __init__(self, input_dim: int, hidden_layers=(32, 16), activation="relu", dropout=0.2):
        super().__init__()
        act = _get_activation(activation)
        layers = []
        prev = input_dim

        for h in hidden_layers:
            layers.append(nn.Linear(prev, h))
            layers.append(act.__class__())
            if dropout and dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev = h

        layers.append(nn.Linear(prev, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def train_mlp(
    X_train_scaled, y_train,
    X_test_scaled,  y_test,
    hidden_layers=(32, 16),
    activation="relu",
    optimizer_name="adam",
    lr=1e-3,
    weight_decay=1e-4,
    dropout=0.2,
    batch_size=64,
    epochs=50,
    seed=0,
    threshold=0.5,
    verbose_every=10,
):

    torch.manual_seed(seed)
    np.random.seed(seed)

    Xtr = torch.tensor(X_train_scaled, dtype=torch.float32)
    ytr = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
    Xte = torch.tensor(X_test_scaled, dtype=torch.float32)

    loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=batch_size, shuffle=True)

    model = MLP(
        input_dim=X_train_scaled.shape[1],
        hidden_layers=hidden_layers,
        activation=activation,
        dropout=dropout
    )

    criterion = nn.BCEWithLogitsLoss()
    optimizer = _get_optimizer(optimizer_name, model.parameters(), lr=lr, weight_decay=weight_decay)

    model.train()
    for epoch in range(epochs):
        running = 0.0
        for xb, yb in loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running += loss.item() * xb.size(0)

        if verbose_every and (epoch + 1) % verbose_every == 0:
            print(f"Epoch {epoch+1}/{epochs} - loss: {running/len(loader.dataset):.4f}")

    model.eval()
    with torch.no_grad():
        logits = model(Xte)
        prob = torch.sigmoid(logits).cpu().numpy().reshape(-1)
        pred = (prob >= threshold).astype(int)

    acc = accuracy_score(y_test, pred)
    ll = log_loss(y_test, prob, labels=[0, 1])

    results = {
        "test_accuracy": acc,
        "test_logloss": ll,
        "prob": prob,
        "pred": pred,
        "model": model
    }
    return results


In [25]:
res1 = train_mlp(
    X_train_scaled, y_train, X_test_scaled, y_test,
    activation="relu",
    optimizer_name="adam",
    hidden_layers=(32,16),
    dropout=0.2,
    lr=1e-3,
    epochs=80,
    batch_size=64,
    weight_decay=1e-4
)
print(res1["test_accuracy"], res1["test_logloss"])


Epoch 10/80 - loss: 0.6590
Epoch 20/80 - loss: 0.6586
Epoch 30/80 - loss: 0.6553
Epoch 40/80 - loss: 0.6560
Epoch 50/80 - loss: 0.6552
Epoch 60/80 - loss: 0.6540
Epoch 70/80 - loss: 0.6536
Epoch 80/80 - loss: 0.6553
0.6372093023255814 0.6401331227221677


In [26]:
res2 = train_mlp(
    X_train_scaled, y_train, X_test_scaled, y_test,
    activation="tanh",
    optimizer_name="adamw",
    hidden_layers=(16,),
    dropout=0.0,
    lr=3e-4,
    epochs=120,
    batch_size=128,
    weight_decay=1e-4
)
print(res2["test_accuracy"], res2["test_logloss"])


Epoch 10/120 - loss: 0.6594
Epoch 20/120 - loss: 0.6557
Epoch 30/120 - loss: 0.6551
Epoch 40/120 - loss: 0.6550
Epoch 50/120 - loss: 0.6548
Epoch 60/120 - loss: 0.6548
Epoch 70/120 - loss: 0.6547
Epoch 80/120 - loss: 0.6547
Epoch 90/120 - loss: 0.6546
Epoch 100/120 - loss: 0.6545
Epoch 110/120 - loss: 0.6545
Epoch 120/120 - loss: 0.6544
0.6381395348837209 0.6380067465068636


In [27]:
res = train_mlp(X_train_scaled, y_train, X_test_scaled, y_test,
    hidden_layers=(16,),
    activation="relu",
    optimizer_name="adam",
    lr=1e-3,
    dropout=0.0,
    weight_decay=1e-4,
    batch_size=128,
    epochs=200
)

print(res["test_accuracy"], res["test_logloss"])

Epoch 10/200 - loss: 0.6553
Epoch 20/200 - loss: 0.6541
Epoch 30/200 - loss: 0.6536
Epoch 40/200 - loss: 0.6533
Epoch 50/200 - loss: 0.6529
Epoch 60/200 - loss: 0.6528
Epoch 70/200 - loss: 0.6527
Epoch 80/200 - loss: 0.6526
Epoch 90/200 - loss: 0.6525
Epoch 100/200 - loss: 0.6522
Epoch 110/200 - loss: 0.6520
Epoch 120/200 - loss: 0.6520
Epoch 130/200 - loss: 0.6518
Epoch 140/200 - loss: 0.6518
Epoch 150/200 - loss: 0.6517
Epoch 160/200 - loss: 0.6516
Epoch 170/200 - loss: 0.6515
Epoch 180/200 - loss: 0.6516
Epoch 190/200 - loss: 0.6514
Epoch 200/200 - loss: 0.6511
0.6311627906976744 0.6379606515839146


In [28]:
res = train_mlp(X_train_scaled, y_train, X_test_scaled, y_test,
    hidden_layers=(32, 16),
    activation="relu",
    optimizer_name="sgd",
    lr=5e-3,
    dropout=0.0,
    weight_decay=1e-4,
    batch_size=128,
    epochs=300
)
print(res["test_accuracy"], res["test_logloss"])

Epoch 10/300 - loss: 0.6569
Epoch 20/300 - loss: 0.6547
Epoch 30/300 - loss: 0.6543
Epoch 40/300 - loss: 0.6539
Epoch 50/300 - loss: 0.6537
Epoch 60/300 - loss: 0.6536
Epoch 70/300 - loss: 0.6535
Epoch 80/300 - loss: 0.6532
Epoch 90/300 - loss: 0.6531
Epoch 100/300 - loss: 0.6531
Epoch 110/300 - loss: 0.6530
Epoch 120/300 - loss: 0.6528
Epoch 130/300 - loss: 0.6526
Epoch 140/300 - loss: 0.6524
Epoch 150/300 - loss: 0.6523
Epoch 160/300 - loss: 0.6523
Epoch 170/300 - loss: 0.6521
Epoch 180/300 - loss: 0.6520
Epoch 190/300 - loss: 0.6518
Epoch 200/300 - loss: 0.6518
Epoch 210/300 - loss: 0.6518
Epoch 220/300 - loss: 0.6516
Epoch 230/300 - loss: 0.6516
Epoch 240/300 - loss: 0.6516
Epoch 250/300 - loss: 0.6514
Epoch 260/300 - loss: 0.6513
Epoch 270/300 - loss: 0.6512
Epoch 280/300 - loss: 0.6510
Epoch 290/300 - loss: 0.6509
Epoch 300/300 - loss: 0.6509
0.6330232558139535 0.6387552698867937
